In [1]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [2]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [3]:
file = 'data/ohlcv_AAVEUSDT_1h_from_1602730800000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2020-10-15 03:00:00,51.430,59.000,46.437,47.099,13042.026,2020-10-15 03:59:59.999000064,18550.125000
1,2020-10-15 04:00:00,47.097,47.916,46.158,47.486,4565.137,2020-10-15 04:59:59.999000064,18550.166667
2,2020-10-15 05:00:00,47.429,49.746,47.428,49.000,5618.395,2020-10-15 05:59:59.999000064,18550.208333
3,2020-10-15 06:00:00,49.000,49.496,48.610,49.031,1977.936,2020-10-15 06:59:59.999000064,18550.250000
4,2020-10-15 07:00:00,49.150,49.208,48.610,49.000,3338.552,2020-10-15 07:59:59.999000064,18550.291667
...,...,...,...,...,...,...,...,...
13527,2022-05-02 12:00:00,140.900,143.200,140.600,142.100,4914.109,2022-05-02 12:59:59.999000064,19114.500000
13528,2022-05-02 13:00:00,142.000,145.500,140.500,144.700,18002.970,2022-05-02 13:59:59.999000064,19114.541667
13529,2022-05-02 14:00:00,144.600,146.400,142.800,145.200,8170.861,2022-05-02 14:59:59.999000064,19114.583333
13530,2022-05-02 15:00:00,145.200,146.200,143.800,144.700,6062.550,2022-05-02 15:59:59.999000064,19114.625000


In [4]:
from app.indicators import find_swings
data_ = find_swings(data)

In [5]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [6]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,59.000,46.437,59.000,0.0,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
1,47.916,46.158,47.916,NaN,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
2,59.000,0.000,59.000,-1.0,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
3,59.000,46.158,46.158,1.0,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
4,49.746,46.158,49.746,-1.0,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527,149.000,144.600,149.000,-1.0,5,False,True,0.0,149.0,0.0,0.0,False,False,False,False
13528,149.000,140.300,140.300,1.0,0,False,False,0.0,0.0,0.0,0.0,False,False,False,False
13529,149.000,140.300,140.300,1.0,1,True,False,140.3,0.0,140.3,0.0,True,False,False,False
13530,149.000,140.300,140.300,1.0,2,True,False,140.3,0.0,0.0,0.0,False,False,False,False


In [7]:
data__=data__*1

In [8]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,59.000,46.437,59.000,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
1,47.916,46.158,47.916,NaN,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
2,59.000,0.000,59.000,-1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
3,59.000,46.158,46.158,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
4,49.746,46.158,49.746,-1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527,149.000,144.600,149.000,-1.0,5,0,1,0.0,149.0,0.0,0.0,0,0,0,0
13528,149.000,140.300,140.300,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
13529,149.000,140.300,140.300,1.0,1,1,0,140.3,0.0,140.3,0.0,1,0,0,0
13530,149.000,140.300,140.300,1.0,2,1,0,140.3,0.0,0.0,0.0,0,0,0,0


In [9]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [10]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [11]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2020-10-15 03:00:00,51.430,59.000,46.437,47.099,13042.026,0,0,0,0
1,2020-10-15 04:00:00,47.097,47.916,46.158,47.486,4565.137,0,0,0,0
2,2020-10-15 05:00:00,47.429,49.746,47.428,49.000,5618.395,0,0,0,0
3,2020-10-15 06:00:00,49.000,49.496,48.610,49.031,1977.936,0,0,0,0
4,2020-10-15 07:00:00,49.150,49.208,48.610,49.000,3338.552,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
13527,2022-05-02 12:00:00,140.900,143.200,140.600,142.100,4914.109,0,0,0,0
13528,2022-05-02 13:00:00,142.000,145.500,140.500,144.700,18002.970,0,0,0,0
13529,2022-05-02 14:00:00,144.600,146.400,142.800,145.200,8170.861,1,0,0,0
13530,2022-05-02 15:00:00,145.200,146.200,143.800,144.700,6062.550,0,0,0,0


In [12]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [13]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2020-10-15 03:00:00,51.430,59.000,46.437,47.099,13042.026,0,0,0,0,1
1,2020-10-15 04:00:00,47.097,47.916,46.158,47.486,4565.137,0,0,0,0,1
2,2020-10-15 05:00:00,47.429,49.746,47.428,49.000,5618.395,0,0,0,0,1
3,2020-10-15 06:00:00,49.000,49.496,48.610,49.031,1977.936,0,0,0,0,1
4,2020-10-15 07:00:00,49.150,49.208,48.610,49.000,3338.552,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
13527,2022-05-02 12:00:00,140.900,143.200,140.600,142.100,4914.109,0,0,0,0,1
13528,2022-05-02 13:00:00,142.000,145.500,140.500,144.700,18002.970,0,0,0,0,1
13529,2022-05-02 14:00:00,144.600,146.400,142.800,145.200,8170.861,1,0,0,0,0
13530,2022-05-02 15:00:00,145.200,146.200,143.800,144.700,6062.550,0,0,0,0,1


In [14]:
df.set_index('o_ts',inplace=True)

In [15]:
df.to_csv('data/transformed_data/AAVEUSDT_1h_transformed.csv')

In [16]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     10441
            1   0       890
    1   0   0   0       858
    0   1   0   0       679
1   0   0   0   0       664
dtype: int64

In [17]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [18]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [19]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [20]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [21]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [22]:
X, y = get_X_y(df, 2000, 21)
print(X.shape)
print(y.shape)

(2000, 20, 10)
(2000, 5)


In [23]:
assert np.isnan(X).sum() == 0

In [24]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [25]:
X_train, y_train = get_X_y(data_train, 2000, 21)
X_test, y_test = get_X_y(data_test, 400, 21)

In [26]:
assert X_train.shape == (2000, 20, 10)

In [27]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [28]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [29]:
model = init_model()
model.summary()

2022-05-06 12:44:38.350287: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 12:44:38.381271: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [31]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [32]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
25/25 [==============================] - 2s 24ms/step - loss: 2.7576 - accuracy: 0.7331 - val_loss: 0.9264 - val_accuracy: 0.7800
Epoch 2/75
25/25 [==============================] - 0s 8ms/step - loss: 2.4540 - accuracy: 0.7100 - val_loss: 1.0254 - val_accuracy: 0.4800
Epoch 3/75
25/25 [==============================] - 0s 9ms/step - loss: 2.3131 - accuracy: 0.5663 - val_loss: 0.9172 - val_accuracy: 0.5775
Epoch 4/75
25/25 [==============================] - 0s 9ms/step - loss: 2.2266 - accuracy: 0.5569 - val_loss: 0.8423 - val_accuracy: 0.6875
Epoch 5/75
25/25 [==============================] - 0s 9ms/step - loss: 2.2024 - accuracy: 0.5806 - val_loss: 0.9016 - val_accuracy: 0.6150
Epoch 6/75
25/25 [==============================] - 0s 8ms/step - loss: 2.1824 - accuracy: 0.5550 - val_loss: 0.8201 - val_accuracy: 0.6725
Epoch 7/75
25/25 [===

In [33]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.9840 - accuracy: 0.4675


[0.9840379953384399, 0.4675000011920929]

In [34]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.04, 0.26, 0.34, 0.35],
       [0.12, 0.12, 0.08, 0.38, 0.3 ],
       ...,
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.24, 0.09, 0.06, 0.28, 0.33],
       [0.  , 0.  , 0.  , 0.  , 1.  ]], dtype=float32)

In [35]:
model.save('./models/RNN_AAVEUSDT_1h.h5')